# First Step
Import the Dataset

In [44]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

data = pd.read_csv("malware analysis.csv")

# drop label columns
X = data.drop(columns=['normal.'])

# isolate y
y = data['normal.']

# split in Train-set(80%) and Testing-set(20%)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20) 



In [45]:
X_train.info(), X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48749 entries, 16274 to 24288
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   0        48749 non-null  int64  
 1   udp      48749 non-null  object 
 2   private  48749 non-null  object 
 3   SF       48749 non-null  object 
 4   105      48749 non-null  int64  
 5   146      48749 non-null  int64  
 6   1        48749 non-null  int64  
 7   1.1      48749 non-null  int64  
 8   1.2      48749 non-null  float64
 9   0.1      48749 non-null  float64
 10  0.2      48749 non-null  float64
 11  255      48749 non-null  int64  
 12  254      48749 non-null  int64  
 13  1.3      48749 non-null  float64
 14  0.01     48749 non-null  float64
 15  0.3      48749 non-null  float64
 16  0.4      48749 non-null  float64
 17  0.5      48749 non-null  float64
dtypes: float64(8), int64(7), object(3)
memory usage: 7.1+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 12188 entries, 22055 to 5

(None, None)

# Second Step 
Clean the dataset and process the data.

In [50]:
def process_data(train, test):
    is_numerical  = np.array( [ len(np.unique(train[col]))>10 for col in train] ) 
    
    numerical_idx = ['105','146','1','1.1','1.2','0.1','0.2','255','254','1.3','0.01','0.3','0.4','0.5']
    # convert numerical to floats (keep NaN)
    X_train = pd.concat([ train[numerical_idx].apply(pd.to_numeric, errors='coerce'), 
                          train[numerical_idx].isna().astype(int)], 
                          axis=1)
    X_train = X_train.fillna(0.0)
    
    # categorical features
    categorical_idx = ['private','SF']
    categorical_idx_ord = ['udp']
    
    oh = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    oh.fit( train[categorical_idx] )
    
    enc = OrdinalEncoder()
    encoded_Ord = enc.fit_transform(train[categorical_idx_ord])
    X_train[categorical_idx_ord] = encoded_Ord
    
    encoded = oh.transform( train[categorical_idx] )
    for i,col in enumerate(oh.get_feature_names_out()):
      X_train[col] = encoded[:,i]
        
    
    X_train.columns = X_train.columns.astype(str)
    
    # --------------
    # process test
    X_test = pd.concat([ test[numerical_idx].apply(pd.to_numeric, errors='coerce'), 
                         test[numerical_idx].isna().astype(int)], 
                           axis=1)
    
    X_test = X_test.fillna(0.0)
    
    encoded_Ord = enc.fit_transform(test[categorical_idx_ord])
    encoded = oh.transform( test[categorical_idx] )
    for i,col in enumerate(oh.get_feature_names_out()):
      X_test[col] = encoded[:,i]
        
    X_test[categorical_idx_ord] = encoded_Ord
    X_test.columns = X_test.columns.astype(str)
    
    return X_train, X_test
    

In [51]:
X_train_enc, X_test_enc = process_data(X_train, X_test)


NotImplementedError: 